<a href="https://colab.research.google.com/github/VivianOuou/NLP-Course/blob/main/course/en/chapter5/section6_Semantic%20search%20with%20FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic search with FAISS (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [6]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

### 翻译
在使用嵌入进行语义搜索的部分中，正如我们在第一章中所见，基于Transformer的语言模型将一段文本中的每个标记（token）表示为一个嵌入向量。事实证明，可以通过“池化”（pooling）这些单独的嵌入来创建整个句子、段落甚至（在某些情况下）文档的向量表示。然后，这些嵌入可以通过计算每个嵌入之间的点积相似度（或其他相似度度量）来找到语料库中相似的文档，并返回重叠度最高的文档。

在本节中，我们将使用嵌入开发一个语义搜索引擎。与基于查询中关键词与文档匹配的传统方法相比，这种搜索引擎具有若干优势。

### 解释
这段文字介绍了一种利用嵌入（embedding）技术进行语义搜索的方法。嵌入是Transformer模型生成的一种向量表示形式，能够捕捉文本的语义信息。通过将一段文本（可以是单词、句子或整篇文档）的多个标记嵌入“池化”成一个统一的向量，可以用这个向量来表示整段文本的含义。

语义搜索的核心是通过比较这些向量之间的相似度（例如使用点积计算）来找到语料库中最相关的文档。这种方法与传统基于关键词匹配的搜索不同，传统搜索只关注是否有相同的词，而语义搜索能理解文本的深层含义。例如，“猫”和“ feline”（猫科动物）在语义上相似，但传统搜索可能无法识别这种关系，而嵌入向量可以捕捉到这种语义联系。

文中提到，这种方法比传统关键词搜索有优势，可能包括更高的准确性、能够处理同义词或语义相近的表达，以及对复杂查询的更好理解。

### 总结
使用嵌入进行语义搜索的基本思路是：
1. 用Transformer模型将文本转化为嵌入向量。
2. 通过“池化”生成整个句子或文档的向量表示。
3. 计算向量间的相似度，找到语料库中最匹配的文档。
4. 这种方法优于传统关键词搜索，因为它基于语义理解，而非简单的词匹配。

### 池化是什么？
“池化”（Pooling）是一种将多个向量组合或聚合为单个向量的技术。在语义搜索的背景下，Transformer模型会为一段文本中的每个标记（token，通常是词或子词）生成一个嵌入向量（embedding vector）。但如果你想表示整个句子、段落或文档的含义，就需要把这些单独的嵌入向量“汇总”成一个统一的向量表示，这个过程就是池化。

常见的池化方法包括：
1. **平均池化（Mean Pooling）**：对所有标记的嵌入向量取平均值。
2. **最大池化（Max Pooling）**：在每个维度上取所有嵌入向量的最大值。
3. **CLS池化**（特定于BERT等模型）：使用模型输出的特殊标记（如 `[CLS]`）的嵌入作为整个句子的表示。

池化的目的是提取一段文本的总体语义特征，减少维度的复杂性，同时保留关键信息。

### 嵌入向量直接计算比较不可以吗？
直接用单个标记的嵌入向量进行比较在理论上是可行的，但实际上存在以下问题：

1. **维度和数量不一致**：
   - 一段文本可能包含多个标记，每个标记都有自己的嵌入向量。如果直接比较两段文本的嵌入向量，问题在于两段文本的标记数量可能不同（比如一个句子有5个词，另一个有10个词），导致无法直接对齐和比较。
   - 池化后，每段文本都被简化为一个固定长度的向量（比如768维，取决于模型），这样就可以方便地进行比较。

2. **语义整体性不足**：
   - 单个标记的嵌入向量只反映该词的局部语义，受限于上下文。如果直接比较这些向量，可能无法准确捕捉整段文本的语义。例如，“I like to run”和“I don’t like to run”中，“like”和“run”的嵌入向量可能相似，但整体意思完全相反。
   - 池化通过综合所有标记的信息，生成一个更能代表整体语义的向量。

3. **计算效率问题**：
   - 如果不池化，而是逐个比较每对标记的嵌入向量（比如用某种相似度度量计算所有组合），计算复杂度会大幅增加，尤其是在处理长文本或大规模语料库时。
   - 池化后，只需计算两个固定向量之间的相似度（比如点积），效率大大提高。

4. **应用场景需求**：
   - 语义搜索的目标通常是比较整个句子或文档的相似性，而不是单个词。直接用标记嵌入比较更适合词级别的任务（如词义消歧），而不是句子或文档级别的语义搜索。

### 总结
池化是将多个嵌入向量整合为一个整体表示的过程，目的是生成固定长度的向量来代表整段文本的语义。直接比较单个嵌入向量在技术上可行，但由于维度不一致、语义不完整、效率低下等问题，通常不适合语义搜索这种需要整体语义比较的任务。因此，池化是语义搜索中不可或缺的一步。

In [7]:
from datasets import load_dataset

issues_dataset = load_dataset("lewtun/github-issues", split="train")
issues_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


datasets-issues-with-comments.jsonl:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

issues_dataset 筛选出的是：

非拉取请求（即普通议题）。
有评论的议题。
这样可以减少噪声，保留对语义搜索最有用的数据。

In [8]:
issues_dataset = issues_dataset.filter(
    lambda x: (x["is_pull_request"] == False and len(x["comments"]) > 0)
)
issues_dataset

Filter:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [9]:
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [10]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]

In [11]:
df["comments"][0].tolist()

['Cool, I think we can do both :)',
 '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).']

这段代码在干什么？

df.explode("comments", ignore_index=True)：
将 df 中的 comments 列从列表展开为单独的行，每行对应一条评论，同时保留其他列的值。
重置索引，确保新 DataFrame 的行号是连续的。
目的是将数据转换为适合生成嵌入的格式，每行是一个独立的 (html_url, title, body, comment) 元组。
comments_df.head(4)：
查看新 DataFrame comments_df 的前 4 行，验证展开操作是否正确。

In [12]:
comments_df = df.explode("comments", ignore_index=True)
comments_df.head(4)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...


In [13]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(comments_df)
comments_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [14]:
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split())}
)

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

In [15]:
comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)
comments_dataset

Filter:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2175
})

In [16]:
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

In [17]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [18]:
import torch

device = torch.device("cuda")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [19]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [20]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [21]:
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [22]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

In [23]:
!pip install faiss-cpu

In [24]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [25]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [26]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [27]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [28]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

COMMENT: Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if offline mode is added similar to how `transformers` loads models offline fine.

@mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could you please elaborate on how that should look like?
SCORE: 25.505016326904297
TITLE: Discussion using datasets in offline mode
URL: https://github.com/huggingface/datasets/issues/824

COMMENT: The local dataset builders (csv, text , json and pandas) are now part of the `datasets` package since #1726 :)
You can now use them offline
```python
datasets = load_dataset('text', data_files=data_files)
```

We'll do a new release soon
SCORE: 24.555540084838867
TITLE: Discussion using datasets in offline mode
URL: https://github.com/huggingface/datasets/issues/824

COMMENT: I opened a PR that allows to reload modules that have already been loaded once even if there's no intern